In [1]:
import pandas as pd

df = pd.read_excel("data/input.xlsx")
df = df.drop(['timestamp'],axis=1)

In [16]:
global_albums_set = set()

def process_entry_row(row):

    row_filled = row.notna()
    row_strip = row[row_filled == True]
    length = row_strip.shape[0] - 1
    # print(length)

    global global_albums_set
    position = 1
    username = row_strip[0]

    entry_list = []

    for entry in row_strip[1:length+1]:

        entry_slug = entry.lower().strip(" ")
        global_albums_set.add(entry_slug)
        score = 25*(length - position)/(length - 1) + 5

        # print("ENTRY: {}".format(entry_slug))
        # print("POSITION: {}".format(position))
        # print("SCORE: {}".format(score))

        entry_list += {
            'username': username,
            'entry': entry_slug,
            'position': position,
            'score': score
        },

        position += 1

    entry_df = pd.DataFrame(entry_list)
    
    return entry_df



In [17]:
entries_df = df.apply(process_entry_row, axis=1)

# print(global_albums_set)

In [18]:
albums = list(global_albums_set)

albums_df = pd.Series(albums)
albums_df.head()

# albums_df.to_excel('data/albums_list.xlsx')

0        diabolizer - khalkedonian death
1                    loharano - loharano
2    borstal - at her majesty’s pleasure
3                          llnn -unmaker
4      proscriptor mcgovern's apsû - s/t
dtype: object

In [19]:
entries_df.shape

(67,)

In [20]:
entries = entries_df.iloc[0]

for i in range(1,entries_df.shape[0]):
    entries = pd.concat([entries,entries_df.iloc[i]])


In [21]:
entries

,username,entry,position,score
0,Moland Fengkov,mastodon – hushed and grim,1,30.000000
1,Moland Fengkov,grey aura – zwart vierkant,2,29.137931
2,Moland Fengkov,papangu – holoceno,3,28.275862
3,Moland Fengkov,godspeed you black emperor – g_d’s pee at stat...,4,27.413793
4,Moland Fengkov,big|brave – vital,5,26.551724
...,...,...,...,...
5,Maxime Fbr,al-namrood - wala'at,6,16.111111
6,Maxime Fbr,kvadrat - ψυχική αποσύνθεση,7,13.333333
7,Maxime Fbr,unctoris - shout demise,8,10.555556
8,Maxime Fbr,ifernach - capitulation of all life,9,7.777778


In [49]:
def build_album_stats(entries):

   aggfunc = {
      'position': ['mean','min','max'],
      'score': ['count','sum','mean','max','min']
   }

   album_stats = pd.pivot_table(entries, index = ['genre','origin','album'], values = ['score','position'], aggfunc=aggfunc)

   album_ranking = album_stats.rank(method='dense',ascending=False)[('score','sum')]
   album_genre_ranking = album_stats.groupby('genre').rank(method='dense',ascending=False)[('score','sum')]
   album_origin_ranking = album_stats.groupby('origin').rank(method='dense',ascending=False)[('score','sum')]

   album_results = pd.merge(album_stats, album_ranking, left_index=True, right_index=True)
   album_results = pd.merge(album_results, album_genre_ranking, left_index=True, right_index=True)
   album_results = pd.merge(album_results, album_origin_ranking, left_index=True, right_index=True)

   album_results.reset_index(inplace=True)

   return album_results

In [23]:
album_entries = pd.read_excel("data/albums_list_complete.xlsx")
album_entries.head()

,entry,album,genre,origin
0,biffy clyro - the myth of the happily ever after,biffy clyro - the myth of the happily ever after,Alternative Rock / Metal,UK
1,glitterer - life is not a lesson,glitterer - life is not a lesson,Alternative Rock / Metal,US
2,greta van fleet - the battle at garden's gate,greta van fleet - the battle at garden's gate,Alternative Rock / Metal,US
3,greta van fleet - the battle at garden’s gate,greta van fleet - the battle at garden's gate,Alternative Rock / Metal,US
4,"passcode - passcode ""strive"" for budokan tour ...","passcode - passcode ""strive"" for budokan tour ...",Alternative Rock / Metal,JP


In [24]:
entries_full = pd.merge(left=entries, right=album_entries, on='entry')

entries_full.head()

,username,entry,position,score,album,genre,origin
0,Moland Fengkov,mastodon – hushed and grim,1,30.000000,mastodon - hushed and grim,Prog Metal,US
1,Moland Fengkov,grey aura – zwart vierkant,2,29.137931,grey aura - zwart vierkant,Post-Black Metal,NL
2,Moland Fengkov,papangu – holoceno,3,28.275862,papangu - holoceno,Prog Metal,BR
3,Moland Fengkov,godspeed you black emperor – g_d’s pee at stat...,4,27.413793,godspeed you black emperor - g_d's pee at stat...,Post-Rock,CA
4,Moland Fengkov,big|brave – vital,5,26.551724,big|brave - vital,Drone Doom - Ritual - Ambient Metal,CA


In [46]:
aggfunc = {
      'position': ['mean','min','max'],
      'score': ['count','sum','mean','max','min']
   }

album_stats = pd.pivot_table(entries_full, index = ['genre','origin','album'], values = ['score','position'], aggfunc=aggfunc)
album_stats.head()

position  \
                                                                                        max   
genre                    origin album                                                         
Alternative Rock / Metal JP     passcode - passcode "strive" for budokan tour 2...     29.0   
                         UK     biffy clyro - the myth of the happily ever after       19.0   
                                tigercub - as blue as indigo                           12.0   
                         US     greta van fleet - the battle at garden's gate           3.0   
                                tomahawk - tonic immobility                            20.0   

                                                                                           \
                                                                                     mean   
genre                    origin album                                                       
Alternative Rock / Metal JP     passcode - passcode "strive" for budokan tour 2...  29.00   
                         UK     biffy clyro - the myth of the happily ever after    18.00   
                                tigercub - as blue as indigo                        12.00   
                         US     greta van fleet - the battle at garden's gate        2.50   
                                tomahawk - tonic immobility                         14.25   

                                                                                          \
                                                                                     min   
genre                    origin album                                                      
Alternative Rock / Metal JP     passcode - passcode "strive" for budokan tour 2...  29.0   
                         UK     biffy clyro - the myth of the happily ever after    17.0   
                                tigercub - as blue as indigo                        12.0   
                         US     greta van fleet - the battle at garden's gate        2.0   
                                tomahawk - tonic immobility                          9.0   

                                                                                   score  \
                                                                                   count   
genre                    origin album                                                      
Alternative Rock / Metal JP     passcode - passcode "strive" for budokan tour 2...     1   
                         UK     biffy clyro - the myth of the happily ever after       2   
                                tigercub - as blue as indigo                           1   
                         US     greta van fleet - the battle at garden's gate          2   
                                tomahawk - tonic immobility                            4   

                                                                                               \
                                                                                          max   
genre                    origin album                                                           
Alternative Rock / Metal JP     passcode - passcode "strive" for budokan tour 2...   5.862069   
                         UK     biffy clyro - the myth of the happily ever after     8.947368   
                                tigercub - as blue as indigo                         7.083333   
                         US     greta van fleet - the battle at garden's gate       28.275862   
                                tomahawk - tonic immobility                         23.103448   

                                                                                               \
                                                                                         mean   
genre                    origin album                                                           
Alternative Rock / Metal JP     pas

In [47]:
album_ranking = album_stats.rank(method='dense',ascending=False)[('score','sum')]

In [48]:
album_genre_ranking = album_stats.groupby('genre').rank(method='dense',ascending=False)[('score','sum')]
album_origin_ranking = album_stats.groupby('origin').rank(method='dense',ascending=False)[('score','sum')]

In [62]:
album_results = build_album_stats(entries_full)

album_results

genre origin  \
                                       
0    Alternative Rock / Metal     JP   
1    Alternative Rock / Metal     UK   
2    Alternative Rock / Metal     UK   
3    Alternative Rock / Metal     US   
4    Alternative Rock / Metal     US   
..                        ...    ...   
624              Thrash Metal     UA   
625              Thrash Metal     US   
626              Thrash Metal     US   
627              Thrash Metal     US   
628              Thrash Metal     US   

                                                 album position             \
                                                            max       mean   
0    passcode - passcode "strive" for budokan tour ...     29.0  29.000000   
1     biffy clyro - the myth of the happily ever after     19.0  18.000000   
2                         tigercub - as blue as indigo     12.0  12.000000   
3        greta van fleet - the battle at garden's gate      3.0   2.500000   
4                          tomahawk - tonic immobility     20.0  14.250000   
..                                                 ...      ...        ...   
624     bestial invasion - the divine comedy : inferno     20.0  10.500000   
625                         exodus - persona non grata     22.0  13.142857   
626            flotsam and jetsam - blood in the water     10.0  10.000000   
627                go ahead and die - go ahead and die     10.0  10.000000   
628                          mordred - the dark parade     27.0  19.000000   

          score_x                                             score_y score  
      min   count        max       mean        min        sum     sum   sum  
0    29.0       1   5.862069   5.862069   5.862069   5.862069   349.0   5.0  
1    17.0       2   8.947368   8.223684   7.500000  16.447368   289.0   3.0  
2    12.0       1   7.083333   7.083333   7.083333   7.083333   343.0   4.0  
3     2.0       2  28.275862  24.971264  21.666667  49.942529   122.0   2.0  
4     9.0       4  23.103448  14.298732   7.380952  57.194927   104.0   1.0  
..    ...     ...        ...        ...        ...        ...     ...   ...  
624   1.0       2  30.000000  20.104167  10.208333  40.208333   160.0   5.0  
625   5.0       7  26.551724  13.167401   5.000000  92.171809    48.0   1.0  
626  10.0       1  15.937500  15.937500  15.937500  15.937500   292.0  12.0  
627  10.0       1   7.500000   7.500000   7.500000   7.500000   340.0  13.0  
628  11.0       2  14.375000  10.980603   7.586207  21.961207   255.0   7.0  

[629 rows x 13 columns]

In [63]:
def format_album_results(album_results):

   album_results.columns = album_results.columns.map('|'.join).str.strip('|')

   map = {
      'score_y|sum':'rank',
      'album_id':'album_id',
      'album':'album',
      'score|sum':'genre_rank',
      'genre_id':'genre_id',
      'genre':'genre',
      'score_x|count':'nb_votes',
      'score_x|sum':'total_score',
      'score_x|mean':'mean_score',
      'score_x|max':'highest_score',
      'score_x|min':'lowest_score',
      'position|mean':'mean_position',
      'position|min':'highest_position',
      'position|max':'lowest_position'
   }

   album_results = album_results.reindex(columns=map.keys())

   drop_cols = (x for x in album_results.columns if x not in map.keys())
   album_results.drop(drop_cols, axis=1, inplace=True)

   album_results.rename(map, axis=1, inplace=True)

   album_results.set_index('album_id', inplace=True, drop=True)

   album_results.sort_values('rank', inplace=True)

   return album_results

In [64]:
def compute_genre_stats(df):

    genre_stats = pd.pivot_table(df, index=['genre'], values=['nb_votes','total_score'], aggfunc=['sum'])
    genre_stats.sort_values(('sum','nb_votes'), ascending=False, inplace=True)

    genre_stats['weight'] = genre_stats[('sum','total_score')] * 100.0 / genre_stats[('sum','total_score')].sum()

    return genre_stats

In [65]:
final_album_results = format_album_results(album_results)
genre_stats = compute_genre_stats(final_album_results)

In [66]:
final_album_results.head(30)

,rank,album,genre_rank,genre_id,genre,nb_votes,total_score,mean_score,highest_score,lowest_score,mean_position,highest_position,lowest_position
album_id,,,,,,,,,,,,,
NaN,1.0,mastodon - hushed and grim,1.0,NaN,Prog Metal,42,1002.352730,23.865541,30.000000,5.000000,5.595238,1.0,21.0
NaN,2.0,archspire - bleed the future,1.0,NaN,Technical Death,17,417.809934,24.577055,29.137931,13.620690,6.764706,2.0,20.0
NaN,3.0,leprous - aphelion,2.0,NaN,Prog Metal,19,372.773745,19.619671,30.000000,5.862069,9.842105,1.0,29.0
NaN,4.0,dordeduh - har,1.0,NaN,Post-Black Metal,18,343.639595,19.091089,30.000000,7.631579,12.055556,1.0,24.0
NaN,5.0,gojira - fortitude,1.0,NaN,Death Metal,18,337.491466,18.749526,30.000000,5.000000,7.666667,1.0,29.0
NaN,6.0,king buffalo - the burden of restleness,1.0,NaN,Stoner - Psych,16,335.564963,20.972810,29.107143,11.034483,10.187500,2.0,23.0
NaN,7.0,grima - rotten garden,1.0,NaN,Black Metal,14,326.760436,23.340031,30.000000,11.034483,8.500000,1.0,23.0
NaN,8.0,vola - witness,3.0,NaN,Prog Metal,13,281.296089,21.638161,28.275862,5.000000,4.384615,2.0,10.0
NaN,9.0,turnstile - glow on,1.0,NaN,Punk,12,275.958539,22.996545,30.000000,13.620690,7.666667,1.0,20.0


In [81]:
top5_by_genre = final_album_results.query("genre_rank < 6.0")
top5_by_genre

,rank,album,genre_rank,genre_id,genre,nb_votes,total_score,mean_score,highest_score,lowest_score,mean_position,highest_position,lowest_position
album_id,,,,,,,,,,,,,
NaN,1.0,mastodon - hushed and grim,1.0,NaN,Prog Metal,42,1002.352730,23.865541,30.000000,5.000000,5.595238,1.0,21.0
NaN,2.0,archspire - bleed the future,1.0,NaN,Technical Death,17,417.809934,24.577055,29.137931,13.620690,6.764706,2.0,20.0
NaN,3.0,leprous - aphelion,2.0,NaN,Prog Metal,19,372.773745,19.619671,30.000000,5.862069,9.842105,1.0,29.0
NaN,4.0,dordeduh - har,1.0,NaN,Post-Black Metal,18,343.639595,19.091089,30.000000,7.631579,12.055556,1.0,24.0
NaN,5.0,gojira - fortitude,1.0,NaN,Death Metal,18,337.491466,18.749526,30.000000,5.000000,7.666667,1.0,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,346.0,lord of the lost - judas,5.0,NaN,Gothic Metal,1,6.315789,6.315789,6.315789,6.315789,19.000000,19.0,19.0
NaN,346.0,ancient mastery - chapter one : across the mou...,3.0,NaN,Dark Synth - Dungeon synth,1,6.315789,6.315789,6.315789,6.315789,19.000000,19.0,19.0
NaN,349.0,"passcode - passcode ""strive"" for budokan tour ...",5.0,NaN,Alternative Rock / Metal,1,5.862069,5.862069,5.862069,5.862069,29.000000,29.0,29.0


In [82]:
final_album_results.to_excel("data/final_album_results.xlsx")
top5_by_genre.to_excel("data/top5_by_genre.xlsx")